In [1]:
!pip install langchain

  Attempting uninstall: langchain-core
    Found existing installation: langchain-core 0.3.0
    Uninstalling langchain-core-0.3.0:
      Successfully uninstalled langchain-core-0.3.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
langchain-openai 0.2.0 requires langchain-core<0.4,>=0.3, but you have langchain-core 0.2.43 which is incompatible.
langchain-openai 0.2.0 requires openai<2.0.0,>=1.40.0, but you have openai 0.28.0 which is incompatible.


In [2]:
!pip install faiss-cpu

In [3]:
!pip install sentence-transformers

In [4]:
!pip install pandas

In [6]:
!pip install tf-keras

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 13.8 MB/s eta 0:00:00


In [7]:
import pandas as pd
from sentence_transformers import SentenceTransformer
import faiss
import numpy as np

In [8]:
df = pd.read_csv("matches.csv")

In [11]:
df.head()

,id,season,city,date,team1,team2,toss_winner,toss_decision,result,dl_applied,winner,win_by_runs,win_by_wickets,player_of_match,venue,umpire1,umpire2,umpire3
0,1,2017,Hyderabad,5/4/2017,Sunrisers Hyderabad,Royal Challengers Bangalore,Royal Challengers Bangalore,field,normal,0,Sunrisers Hyderabad,35,0,Yuvraj Singh,"Rajiv Gandhi International Stadium, Uppal",AY Dandekar,NJ Llong,NaN
1,2,2017,Pune,6/4/2017,Mumbai Indians,Rising Pune Supergiant,Rising Pune Supergiant,field,normal,0,Rising Pune Supergiant,0,7,SPD Smith,Maharashtra Cricket Association Stadium,A Nand Kishore,S Ravi,NaN
2,3,2017,Rajkot,7/4/2017,Gujarat Lions,Kolkata Knight Riders,Kolkata Knight Riders,field,normal,0,Kolkata Knight Riders,0,10,CA Lynn,Saurashtra Cricket Association Stadium,Nitin Menon,CK Nandan,NaN
3,4,2017,Indore,8/4/2017,Rising Pune Supergiant,Kings XI Punjab,Kings XI Punjab,field,normal,0,Kings XI Punjab,0,6,GJ Maxwell,Holkar Cricket Stadium,AK Chaudhary,C Shamshuddin,NaN
4,5,2017,Bangalore,8/4/2017,Royal Challengers Bangalore,Delhi Daredevils,Royal Challengers Bangalore,bat,normal,0,Royal Challengers Bangalore,15,0,KM Jadhav,M Chinnaswamy Stadium,NaN,NaN,NaN


In [12]:
df.shape

(636, 18)

In [14]:
df = df.drop('umpire3',axis=1)

In [15]:
df.shape

(636, 17)

In [16]:
df = df.drop('id',axis=1)

In [17]:
df.shape

(636, 16)

In [19]:
df.columns

Index(['season', 'city', 'date', 'team1', 'team2', 'toss_winner',
       'toss_decision', 'result', 'dl_applied', 'winner', 'win_by_runs',
       'win_by_wickets', 'player_of_match', 'venue', 'umpire1', 'umpire2'],
      dtype='object')

In [20]:
df.isnull().sum()

season             0
city               7
date               0
team1              0
team2              0
toss_winner        0
toss_decision      0
result             0
dl_applied         0
winner             3
win_by_runs        0
win_by_wickets     0
player_of_match    3
venue              0
umpire1            1
umpire2            1
dtype: int64

In [21]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 636 entries, 0 to 635
Data columns (total 16 columns):
 #   Column           Non-Null Count  Dtype 
---  ------           --------------  ----- 
 0   season           636 non-null    int64 
 1   city             629 non-null    object
 2   date             636 non-null    object
 3   team1            636 non-null    object
 4   team2            636 non-null    object
 5   toss_winner      636 non-null    object
 6   toss_decision    636 non-null    object
 7   result           636 non-null    object
 8   dl_applied       636 non-null    int64 
 9   winner           633 non-null    object
 10  win_by_runs      636 non-null    int64 
 11  win_by_wickets   636 non-null    int64 
 12  player_of_match  633 non-null    object
 13  venue            636 non-null    object
 14  umpire1          635 non-null    object
 15  umpire2          635 non-null    object
dtypes: int64(4), object(12)
memory usage: 79.6+ KB


In [22]:
columns_to_fill = ["umpire1", "umpire2", "player_of_match", "winner", "city"]

for column in columns_to_fill:
        mode_value = df[column].mode()[0]
        df[column].fillna(mode_value, inplace=True)

/var/folders/x3/6tds_w9j0lb0pr8y5qsz97tm0000gn/T/ipykernel_59765/1336196659.py:5: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df[column].fillna(mode_value, inplace=True)


In [23]:
df.isnull().sum()

season             0
city               0
date               0
team1              0
team2              0
toss_winner        0
toss_decision      0
result             0
dl_applied         0
winner             0
win_by_runs        0
win_by_wickets     0
player_of_match    0
venue              0
umpire1            0
umpire2            0
dtype: int64

In [24]:
df.to_csv('cleaned_data.csv')

In [25]:
df = pd.read_csv('cleaned_data.csv')

In [26]:
def row_to_text(row):
    text = (
        f"Season: {row['season']}, City: {row['city']}, Date: {row['date']}, "
        f"Team 1: {row['team1']}, Team 2: {row['team2']}, Toss Winner: {row['toss_winner']}, "
        f"Toss Decision: {row['toss_decision']}, Result: {row['result']}, DL Applied: {row['dl_applied']}, "
        f"Winner: {row['winner']}, Win by Runs: {row['win_by_runs']}, Win by Wickets: {row['win_by_wickets']}, "
        f"Player of the Match: {row['player_of_match']}, Venue: {row['venue']}, Umpire 1: {row['umpire1']}, "
        f"Umpire 2: {row['umpire2']}"
    )
    return text

In [27]:
df['combined_text'] = df.apply(row_to_text, axis=1)

In [32]:
df['combined_text'][0]

'Season: 2017, City: Hyderabad, Date: 5/4/2017, Team 1: Sunrisers Hyderabad, Team 2: Royal Challengers Bangalore, Toss Winner: Royal Challengers Bangalore, Toss Decision: field, Result: normal, DL Applied: 0, Winner: Sunrisers Hyderabad, Win by Runs: 35, Win by Wickets: 0, Player of the Match: Yuvraj Singh, Venue: Rajiv Gandhi International Stadium, Uppal, Umpire 1: AY Dandekar, Umpire 2: NJ Llong'

In [33]:
df['combined_text'].to_csv('textdata.txt',index=False,header=False)

In [35]:
file_path = "textdata.txt"
with open(file_path, "r") as file:
    content = file.read()
content = content.replace('"', '')
with open(file_path, "w") as file:
    file.write(content)

In [37]:
!pip install langchain-ollama

  Attempting uninstall: langsmith
    Found existing installation: langsmith 0.1.121
    Uninstalling langsmith-0.1.121:
      Successfully uninstalled langsmith-0.1.121
  Attempting uninstall: langchain-core
    Found existing installation: langchain-core 0.2.43
    Uninstalling langchain-core-0.2.43:
      Successfully uninstalled langchain-core-0.2.43
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
langchain-google-genai 1.0.10 requires langchain-core<0.3,>=0.2.33, but you have langchain-core 0.3.28 which is incompatible.
langchain-community 0.2.16 requires langchain-core<0.3.0,>=0.2.38, but you have langchain-core 0.3.28 which is incompatible.
langchain-community 0.2.16 requires langsmith<0.2.0,>=0.1.0, but you have langsmith 0.2.6 which is incompatible.
langchain-openai 0.2.0 requires openai<2.0.0,>=1.40.0, but you have openai 0.28.0 which is incompatible.

In [39]:
!pip install --upgrade langchain langchain-core

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 10.4 MB/s eta 0:00:00
  Attempting uninstall: langchain-text-splitters
    Found existing installation: langchain-text-splitters 0.2.4
    Uninstalling langchain-text-splitters-0.2.4:
      Successfully uninstalled langchain-text-splitters-0.2.4
  Attempting uninstall: langchain
    Found existing installation: langchain 0.2.16
    Uninstalling langchain-0.2.16:
      Successfully uninstalled langchain-0.2.16
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
langchain-community 0.2.16 requires langchain<0.3.0,>=0.2.16, but you have langchain 0.3.13 which is incompatible.
langchain-community 0.2.16 requires langchain-core<0.3.0,>=0.2.38, but you have langchain-core 0.3.28 which is incompatible.
langchain-community 0.2.16 requires langsmith<0.2.0,>=0.1.0, but you have langsmith 0.2.6 which is incompatible.


In [43]:
!pip uninstall langchain langchain-core langchain-ollama -y

Found existing installation: langchain 0.3.13
Uninstalling langchain-0.3.13:
  Successfully uninstalled langchain-0.3.13
Found existing installation: langchain-core 0.3.28
Uninstalling langchain-core-0.3.28:
  Successfully uninstalled langchain-core-0.3.28
Found existing installation: langchain-ollama 0.2.2
Uninstalling langchain-ollama-0.2.2:
  Successfully uninstalled langchain-ollama-0.2.2


In [44]:
!pip cache purge

Files removed: 3937


In [63]:
!pip install langchain-core==0.1.18
!pip install langchain-community==0.0.13
!pip install langchain==0.1.4
!pip install langchain-ollama==0.0.4

  Attempting uninstall: packaging
    Found existing installation: packaging 24.1
    Uninstalling packaging-24.1:
      Successfully uninstalled packaging-24.1
  Attempting uninstall: langsmith
    Found existing installation: langsmith 0.1.147
    Uninstalling langsmith-0.1.147:
      Successfully uninstalled langsmith-0.1.147
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
langchain-text-splitters 0.3.4 requires langchain-core<0.4.0,>=0.3.26, but you have langchain-core 0.1.18 which is incompatible.
langchain-google-genai 1.0.10 requires langchain-core<0.3,>=0.2.33, but you have langchain-core 0.1.18 which is incompatible.
langchain-openai 0.2.0 requires langchain-core<0.4,>=0.3, but you have langchain-core 0.1.18 which is incompatible.
langchain-openai 0.2.0 requires openai<2.0.0,>=1.40.0, but you have openai 0.28.0 which is incompatible.
   ━━━━━━━━━━━━━━━

In [70]:
# First, uninstall all related packages
!pip uninstall -y langchain langchain-core langchain-community langchain-ollama langchain-google-genai langchain-openai langsmith

# Then install them in order with compatible versions
!pip install "langchain-core>=0.3.28"
!pip install "langchain-community>=0.0.20"
!pip install "langchain>=0.1.4"
!pip install "langchain-ollama>=0.2.2"

Found existing installation: langchain 0.1.4
Uninstalling langchain-0.1.4:
  Successfully uninstalled langchain-0.1.4
Found existing installation: langchain-core 0.3.28
Uninstalling langchain-core-0.3.28:
  Successfully uninstalled langchain-core-0.3.28
Found existing installation: langchain-community 0.0.20
Uninstalling langchain-community-0.0.20:
  Successfully uninstalled langchain-community-0.0.20
Found existing installation: langchain-ollama 0.2.2
Uninstalling langchain-ollama-0.2.2:
  Successfully uninstalled langchain-ollama-0.2.2
Found existing installation: langchain-google-genai 1.0.10
Uninstalling langchain-google-genai-1.0.10:
  Successfully uninstalled langchain-google-genai-1.0.10
Found existing installation: langchain-openai 0.2.0
Uninstalling langchain-openai-0.2.0:
  Successfully uninstalled langchain-openai-0.2.0
Found existing installation: langsmith 0.2.6
Uninstalling langsmith-0.2.6:
  Successfully uninstalled langsmith-0.2.6
  Using cached langchain_core-0.3.28-py

In [64]:
from langchain.llms import Ollama

In [77]:
from langchain.vectorstores import FAISS
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.chains import RetrievalQA
from langchain.llms import HuggingFaceHub
from langchain.text_splitter import CharacterTextSplitter
from langchain_ollama import OllamaLLM
from langchain.chains import LLMChain

ImportError: cannot import name 'ToolOutputMixin' from 'langchain_core.messages.tool' (/opt/anaconda3/lib/python3.11/site-packages/langchain_core/messages/tool.py)

In [75]:
!pip uninstall langchain-openai -y
!pip install langchain-openai

  Attempting uninstall: openai
    Found existing installation: openai 0.28.0
    Uninstalling openai-0.28.0:
      Successfully uninstalled openai-0.28.0


In [78]:
# Read and preprocess the text file
with open("textdata.txt", "r") as file:
    raw_text = file.read()

# Split the text based on `Match ID` occurrences
preprocessed_text = raw_text.replace("Season:", "\nSeason:")

# Save the preprocessed text to a new file
with open("textdata.txt", "w") as file:
    file.write(preprocessed_text)


In [79]:
# Read and preprocess the text file
with open("textdata.txt", "r") as file:
    text = file.read()


In [80]:
text_splitter = CharacterTextSplitter(
    chunk_size=500,  
    chunk_overlap=50
)
text_chunks = text_splitter.split_text(text)

In [81]:
# Verify the text splitting process
text_chunks = text_splitter.split_text(text)

print(f"Number of chunks created: {len(text_chunks)}")
for i, chunk in enumerate(text_chunks[:5]):  # Display the first 5 chunks
    print(f"Chunk {i+1}:\n{chunk}\n{'-'*50}")

Number of chunks created: 636
Chunk 1:
Season: 2017, City: Hyderabad, Date: 5/4/2017, Team 1: Sunrisers Hyderabad, Team 2: Royal Challengers Bangalore, Toss Winner: Royal Challengers Bangalore, Toss Decision: field, Result: normal, DL Applied: 0, Winner: Sunrisers Hyderabad, Win by Runs: 35, Win by Wickets: 0, Player of the Match: Yuvraj Singh, Venue: Rajiv Gandhi International Stadium, Uppal, Umpire 1: AY Dandekar, Umpire 2: NJ Llong
--------------------------------------------------
Chunk 2:
Season: 2017, City: Pune, Date: 6/4/2017, Team 1: Mumbai Indians, Team 2: Rising Pune Supergiant, Toss Winner: Rising Pune Supergiant, Toss Decision: field, Result: normal, DL Applied: 0, Winner: Rising Pune Supergiant, Win by Runs: 0, Win by Wickets: 7, Player of the Match: SPD Smith, Venue: Maharashtra Cricket Association Stadium, Umpire 1: A Nand Kishore, Umpire 2: S Ravi
--------------------------------------------------
Chunk 3:
Season: 2017, City: Rajkot, Date: 7/4/2017, Team 1: Gujarat Lio

In [83]:
!pip install -U langchain-huggingface

In [84]:
embeddings_model = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")

In [91]:
faiss_index = FAISS.from_texts(texts=text_chunks, embedding=embeddings_model)

In [89]:
from langchain.prompts import PromptTemplate
from langchain.vectorstores import FAISS

In [121]:
prompt_template = """
You are a cricket analytics expert specializing in IPL analysis. Analyze the provided match data and answer questions with precision.

Data Structure Understanding:
Each data chunk contains the following fields:
- Season: The IPL season year
- City: Location of the match
- Date: In DD/MM/YYYY format
- Team 1 & Team 2: The competing teams
- Toss Winner & Toss Decision: Team winning the toss and their choice
- Result: Match result type (normal/tied/no result)
- DL Applied: Whether Duckworth-Lewis method was used (0/1)
- Winner: The winning team
- Win by Runs: Margin of victory for team batting first
- Win by Wickets: Margin of victory for team batting second
- Player of the Match: Outstanding player of the game
- Venue: Specific stadium name
- Umpire 1 & Umpire 2: Match officials


Response Guidelines:

1. Match Details:
   - Always mention the specific date and season when referring to matches
   - Use full venue names as provided in the data
   - Specify city location when relevant
   - Include toss details when discussing match context

2. Team Performance:
   - Mention batting order (Team 1 vs Team 2) when relevant
   - Specify victory margins accurately (runs/wickets)
   - Include toss impact if pertinent to the query
   - Note if DL method was applied

3. Player Achievements:
   - Always mention Player of the Match awards
   - Connect player performances to match outcomes
   - Include relevant match context

4. Statistical Analysis:
   - Provide exact numbers as shown in the data
   - Include the complete context (venue, season, teams)
   - Specify if data spans multiple chunks
   - Maintain chronological order when discussing multiple matches

Error Handling:
1. If information is not found in the chunks:
   - Clearly state that the specific information is not present in the provided context
   - Do not make assumptions about missing data
   - Provide any relevant partial information from available chunks

2. For team name queries:
   - Recognize both full names and common abbreviations
   - Common mappings:
     - RCB = Royal Challengers Bangalore
     - CSK = Chennai Super Kings
     - KKR = Kolkata Knight Riders
     - MI = Mumbai Indians
     - KXIP/PBKS = Kings XI Punjab/Punjab Kings
     - DD/DC = Delhi Daredevils/Delhi Capitals
     - SRH = Sunrisers Hyderabad
     - RR = Rajasthan Royals

Context:
{context}

Question:
{question}

Answer: Let me analyze the match data and provide a precise response based on the available information.
"""

prompt = PromptTemplate(
    input_variables=["context", "question"],
    template=prompt_template,
    validate_template=True
)

In [92]:
retriever = faiss_index.as_retriever()

In [102]:
!ollama pull phi3

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


pulling manifest ⠋ pulling manifest ⠙ pulling manifest ⠹ pulling manifest ⠸ pulling manifest ⠼ pulling manifest ⠴ pulling manifest ⠦ pulling manifest ⠧ pulling manifest ⠇ pulling manifest ⠏ pulling manifest ⠋ pulling manifest ⠙ pulling manifest ⠹ pulling manifest ⠸ pulling manifest ⠼ pulling manifest ⠴ pulling manifest ⠦ pulling manifest ⠧ pulling manifest ⠇ pulling manifest ⠏ pulling manifest ⠙ pulling manifest ⠙ pulling manifest ⠹ pulling manifest 
pulling 633fc5be925f...   0% ▕                ▏    0 B/2.2 GB                  pulling manifest 
pulling 633fc5be925f...   0% ▕                ▏    0 B/2.2 GB                  pulling manifest 
pulling 633fc5be925f...   0% ▕                ▏    0 B/2.2 GB                  pulling manifest 
pulling 633fc5be925f...   0% ▕                ▏    0 B/2.2 GB                  pulling manifest 
pulling 633fc5be925f...   0% ▕                ▏    0 B/2.2 GB                  pulling manifest 
pulling 633fc5be925f...   0% ▕                ▏    0 B/2.2 G

In [110]:
from langchain.llms import Ollama

# Initialize Mistral
llm = Ollama(
    model="phi3",
    base_url="http://localhost:11434",
    temperature=0.3
)

In [100]:
#llm = OllamaLLM(base_url="http://localhost:11434", model="phi3")

In [104]:
def answer_with_context(query):
    retriever = faiss_index.as_retriever(search_type="similarity", search_kwargs={"k": 10})
    docs = retriever.get_relevant_documents(query)
    context = "\n".join([doc.page_content for doc in docs])
    chain = LLMChain(llm=llm, prompt=prompt)
    response = chain.run({"context": context, "question": query})
    return response, context

In [105]:
qa_chain = RetrievalQA.from_chain_type(llm=llm, retriever=retriever)

In [118]:
questions = [
    "1. Which two teams played an IPL match on the 5th of April 2017 ?",
    "2. Who won the toss during the match that happened on the 7th of April 2017 ?",
    "3. Who won the match between KKR and RR that happened on 20/05/08 ?",
    "4.  How many matches happened on the 9th of April 2017 ?",
    "5. In what locations did KKR and CSK play a match in the year 2010 ?",
    "6. What was the highest margin of victory (by runs) achieved in matches played at Wankhede Stadium in 2017?",
    "7. How many matches did MI play against CSK in the 2017 season, and what were the results?",
    "8. Who were the umpires for the match where the team batting second won by the smallest margin in 2017?",
    "9. Which team had the most wins in matches played at Eden Gardens across all seasons?",
    "10 . What was the total number of no-results (matches without a winner) in IPL history up to the 2017 season?",
]

In [119]:
from langchain.chains import LLMChain

In [125]:
for i in questions[6:] : 
    response, context = answer_with_context(i)
    print("\nQuery:", i)
    print("\nRetrieved Context:\n", context)
    print("\nGenerated Response:\n", response)
    break


Query: 7. How many matches did MI play against CSK in the 2017 season, and what were the results?

Retrieved Context:
 Season: 2017, City: Rajkot, Date: 18/04/17, Team 1: Royal Challengers Bangalore, Team 2: Gujarat Lions, Toss Winner: Gujarat Lions, Toss Decision: field, Result: normal, DL Applied: 0, Winner: Royal Challengers Bangalore, Win by Runs: 21, Win by Wickets: 0, Player of the Match: CH Gayle, Venue: Saurashtra Cricket Association Stadium, Umpire 1: S Ravi, Umpire 2: VK Sharma
Season: 2017, City: Rajkot, Date: 14/04/17, Team 1: Rising Pune Supergiant, Team 2: Gujarat Lions, Toss Winner: Gujarat Lions, Toss Decision: field, Result: normal, DL Applied: 0, Winner: Gujarat Lions, Win by Runs: 0, Win by Wickets: 7, Player of the Match: AJ Tye, Venue: Saurashtra Cricket Association Stadium, Umpire 1: A Nand Kishore, Umpire 2: S Ravi
Season: 2017, City: Rajkot, Date: 23/04/17, Team 1: Kings XI Punjab, Team 2: Gujarat Lions, Toss Winner: Gujarat Lions, Toss Decision: field, Result: